# Bibs

In [1]:
import os
import re
from pathlib import Path
import fitz
import sys
import shutil
import unicodedata
from typing import List, Tuple
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
tqdm.pandas()

/home/julio/Documentos/agents_pos/amldo/AMLDO/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extract data
- PDF -> TXT

In [2]:
def processar_pdf_lei(pdf_path: Path, output_dir: Path) -> Path | None:
    """
    Extrai texto de um PDF de lei, limpa headers/footers dinamicamente
    e adiciona marcadores estruturais.
    
    Salva o arquivo .txt no diretório de saída e retorna o caminho.
    Retorna None em caso de falha.
    """
    
    # --- 1. Configuração Dinâmica ---
    
    # Extrai o número base do nome do arquivo (ex: "14133" de "L14133.pdf")
    file_stem = pdf_path.stem
    law_number_match = re.search(r'\d+', file_stem)
    
    if law_number_match:
        # Usa o primeiro grupo de números encontrado (ex: "14133", "13709", "10024", "123")
        law_number_str = law_number_match.group(0)
    else:
        # Fallback caso não encontre números (ex: "minha_lei.pdf" -> "minha_lei")
        law_number_str = re.escape(file_stem)

    # Define o caminho de saída
    output_dir.mkdir(parents=True, exist_ok=True)
    txt_output_path = output_dir / (pdf_path.stem + ".txt")

    # PADRÕES A REMOVER (header/footer)
    RE_DATE_TIME   = re.compile(r"^\s*\d{2}/\d{2}/\d{4},\s*\d{2}:\d{2}\s*$")
    RE_URL         = re.compile(r"^\s*https?://www\.planalto\.gov\.br/.*$", re.I)
    # Regex dinâmico: corresponde a "L 14133", "D10024", "123", etc.
    RE_LAWCODE     = re.compile(r"^\s*[A-Z]*\s*" + re.escape(law_number_str) + r"\s*$", re.I)
    RE_PAGECOUNT   = re.compile(r"^\s*\d+\s*/\s*\d+\s*$")
    
    REMOVE_LINE_IF_MATCH = [RE_DATE_TIME, RE_URL, RE_LAWCODE, RE_PAGECOUNT]

    def is_header_footer(line: str) -> bool:
        s = line.strip()
        # Usa a lista de regex definida dentro da função 'processar_pdf_lei'
        return any(p.match(s) for p in REMOVE_LINE_IF_MATCH)

    # DETECTORES HIERÁRQUICOS
    RE_TITLE    = re.compile(r"^\s*T[ÍI]TULO\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_CHAPTER  = re.compile(r"^\s*CAP[ÍI]TULO\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_SECTION  = re.compile(r"^\s*(SEÇÃO|SECAO)\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_ARTICLE  = re.compile(r"^\s*Art\.\s*(\d+)[\s\.-].*$")
    RE_PARAGRAPH= re.compile(r"^\s*§\s*(\d+)[ºo]?\b")

    # DETECTORES HIERÁRQUICOS
    RE_TITLE      = re.compile(r"^\s*T[ÍI]TULO\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_CHAPTER    = re.compile(r"^\s*CAP[ÍI]TULO\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_SECTION    = re.compile(r"^\s*(SEÇÃO|SECAO)\s+[IVXLCDM]+(\b.*)?$", re.I)
    RE_SUBSECTION = re.compile(r"^\s*SUBSE[ÇC][ÃA]O\s+[IVXLCDM]+(\b.*)?$", re.I)

    # Artigos do tipo: "Art. 5º", "Art 5º", "Art. 5º-A", "Art. 37-A", etc.
    # Captura o número com sufixos simples (A, B, -A) e ignora pontuação depois.
    RE_ARTICLE    = re.compile(
        r"^\s*Art\.?\s*(\d+(?:[A-Za-z]|-\s*[A-Za-z])?)\s*[ºo]?\b.*$", re.I
    )

    # Parágrafos: "§ 1º", "§1º", etc. (mantido)
    RE_PARAGRAPH  = re.compile(r"^\s*§\s*(\d+)[ºo]?\b", re.I)

    def maybe_mark_structural(line: str, out: list[str]) -> bool:
        """Insere marcadores estruturais se a linha corresponder."""
        s = line.strip()

        if RE_TITLE.match(s):
            out.append("")                 # separador visual p/ split
            out.append(f"[[TITLE: {s}]]")
            return True

        if RE_CHAPTER.match(s):
            out.append("")                 # separador visual p/ split
            out.append(f"[[CHAPTER: {s}]]")
            return True

        if RE_SECTION.match(s):
            out.append("")                 # manter padrão de separador
            out.append(f"[[SECTION: {s}]]")
            return True

        if RE_SUBSECTION.match(s):
            out.append("")                 # novo nível: SUBSEÇÃO
            out.append(f"[[SUBSECTION: {s}]]")
            return True

        m_art = RE_ARTICLE.match(s)
        if m_art:
            art_num = m_art.group(1).replace(" ", "")  # normaliza " - A" -> "-A"
            out.append("")                 # separador macro
            out.append(f"[[ARTICLE: {art_num}]]")
            out.append(s)                  # mantém a linha original do artigo
            return True

        m_par = RE_PARAGRAPH.match(s)
        if m_par:
            out.append(f"[[PARAGRAPH: §{m_par.group(1)}]]")
            out.append(s)
            return True

        return False

    # UTILS
    def clean_spaces(s: str) -> str:
        s = s.replace("\xa0", " ")
        s = re.sub(r"[ \t]+", " ", s)
        return s.rstrip()

    def unhyphenate_softbreaks(text: str) -> str:
        """Junta palavras quebradas por hífen no fim da linha."""
        return re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)

    # --- 3. PIPELINE DE EXECUÇÃO ---
    
    if not pdf_path.exists():
        print(f"Erro: Arquivo PDF não encontrado em: {pdf_path}", file=sys.stderr)
        return None

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"Erro ao abrir PDF {pdf_path}: {e}", file=sys.stderr)
        return None
        
    out_lines: list[str] = []
    print(f"Processando {doc.page_count} páginas de {pdf_path.name}...")

    for page in doc:
        page_text = page.get_text()
        
        for ln in page_text.split('\n'):
            ln_clean = clean_spaces(ln)
            
            if not ln_clean:
                continue
            # A função is_header_footer agora usa o RE_LAWCODE dinâmico
            if is_header_footer(ln_clean):
                continue
            
            if not maybe_mark_structural(ln_clean, out_lines):
                out_lines.append(ln_clean)

    doc.close()
    
    text = "\n".join(out_lines)
    text = unhyphenate_softbreaks(text)
    text = re.sub(r"\n{3,}", "\n\n", text) # Consolida quebras de linha
    final_text = text.strip()
    
    # Salva o arquivo de texto final
    try:
        txt_output_path.write_text(final_text, encoding="utf-8")
        print(f"✔️ Salvo (simplificado): {txt_output_path}")
        return txt_output_path # Retorna o caminho do arquivo salvo
    except Exception as e:
        print(f"Erro ao salvar arquivo {txt_output_path}: {e}", file=sys.stderr)
        return None


INPUT_DIR = Path("data/raw")
OUTPUT_DIR = Path("data/text")

pdf_files_a_processar = [
    INPUT_DIR / 'L14133.pdf',
    INPUT_DIR / 'L13709.pdf',
    INPUT_DIR / 'D10024.pdf',
    INPUT_DIR / 'Lcp123.pdf'
]

print("--- Iniciando processamento em lote ---")

arquivos_salvos = []
for pdf_file in pdf_files_a_processar:
    caminho_salvo = processar_pdf_lei(pdf_path=pdf_file, output_dir=OUTPUT_DIR)
    if caminho_salvo:
        arquivos_salvos.append(caminho_salvo)
    print("-" * 20) # Separador

print(f"\n--- Processamento concluído ---")
print(f"Total de arquivos salvos: {len(arquivos_salvos)}")

--- Iniciando processamento em lote ---
Processando 73 páginas de L14133.pdf...
✔️ Salvo (simplificado): data/text/L14133.txt
--------------------
Processando 26 páginas de L13709.pdf...
✔️ Salvo (simplificado): data/text/L13709.txt
--------------------
Processando 15 páginas de D10024.pdf...
✔️ Salvo (simplificado): data/text/D10024.txt
--------------------
Processando 43 páginas de Lcp123.pdf...
✔️ Salvo (simplificado): data/text/Lcp123.txt
--------------------

--- Processamento concluído ---
Total de arquivos salvos: 4


# Tratamento e Splits

In [3]:
SPLIT_ROOT = Path("data/split_docs")

def slugify_path(text: str, max_len: int = 120) -> str:
    """
    Gera uma pasta segura a partir do [[TITLE: ...]]:
      - remove acentos, troca separadores/ilegais por '_', limita tamanho.
    """
    nfkd = unicodedata.normalize("NFKD", text)
    ascii_text = "".join(ch for ch in nfkd if not unicodedata.combining(ch))
    ascii_text = ascii_text.replace("[[TITLE:", "").replace("]]", "").strip()

    ascii_text = ascii_text.replace(os.sep, "_").replace("\\", "_").replace("/", "_")
    ascii_text = re.sub(r"[:*?\"<>|]", "_", ascii_text)  # caracteres reservados (Windows)
    ascii_text = re.sub(r"\s+", " ", ascii_text).strip().replace(" ", "_")
    ascii_text = re.sub(r"_+", "_", ascii_text)

    if len(ascii_text) > max_len:
        ascii_text = ascii_text[:max_len].rstrip("_")

    return ascii_text or "TITULO"

def split_by_titles_with_preamble(full_text: str) -> List[Tuple[str, str]]:
    """
    Divide exclusivamente por [[TITLE: ...]], mas preserva o texto ANTES do primeiro título
    como ('TITULO_0', <conteudo_pre_titulo>).
    Se não houver nenhum título, retorna apenas ('TITULO_0', texto inteiro).
    Retorna lista na ordem: [ (id_ou_titulo, conteudo) ]
      - Primeiro elemento pode ser ('TITULO_0', ...) se houver preâmbulo.
      - Demais elementos usam o título completo como chave.
    """
    lines = full_text.splitlines()
    preamble_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []  # (key, lines)
    current_title: str | None = None
    current_lines: List[str] = []

    found_any_title = False

    for line in lines:
        if line.startswith("[[TITLE:"):
            if not found_any_title:
                # Primeira ocorrência de título: guarda preâmbulo (se existir)
                pre_text = "\n".join(preamble_lines).strip()
                if pre_text:
                    parts.append(("TITULO_0", [pre_text]))
                found_any_title = True

            # Fechar bloco do título anterior, se houver
            if current_title is not None:
                parts.append((current_title, current_lines))

            current_title = line.strip()  # guarda o título completo
            current_lines = []
        else:
            if not found_any_title:
                # Ainda no preâmbulo (antes do primeiro [[TITLE:)
                preamble_lines.append(line)
            else:
                # Dentro de um título já aberto
                if current_title is not None:
                    current_lines.append(line)

    # Fecha o último bloco
    if current_title is not None:
        parts.append((current_title, current_lines))

    # Se nenhum [[TITLE:]] encontrado, tudo é TÍTULO 0
    if not found_any_title:
        only = full_text.strip()
        return [("TITULO_0", only)]

    # Converte lines -> str
    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        if isinstance(ls, list):
            content = "\n".join(ls).strip()
        else:
            content = str(ls).strip()
        if content:
            normalized.append((key, content))
        else:
            # Se for um título sem conteúdo, ainda assim preserva vazio
            normalized.append((key, ""))

    return normalized

def salvar_splits_por_titulo(txt_path: Path, split_root: Path = SPLIT_ROOT) -> List[Path]:
    """
    Lê o .txt (gerado por processar_pdf_lei), divide por TÍTULOS e preâmbulo (TÍTULO 0),
    e salva em: split_root/<PDF_STEM>/
      - TITULO_0/titulo_0.txt (se houver preâmbulo ou nenhum título)
      - <PASTA_TITULO_K>/titulo_k.txt para k >= 1
    """
    if not txt_path.exists():
        print(f"[split] Arquivo não encontrado: {txt_path}", file=sys.stderr)
        return []

    split_root.mkdir(parents=True, exist_ok=True)

    # Pasta do arquivo (um diretório por PDF de origem)
    file_dir = split_root / txt_path.stem
    # Limpa para refazer sempre do zero
    if file_dir.exists():
        shutil.rmtree(file_dir)
    file_dir.mkdir(parents=True, exist_ok=True)

    full_text = txt_path.read_text(encoding="utf-8")
    blocks = split_by_titles_with_preamble(full_text)

    saved: List[Path] = []
    # Índices: 0 para preâmbulo (TITULO_0), depois 1..n para os títulos
    idx = 0
    for key, content in blocks:
        if key == "TITULO_0":
            titulo_dir = file_dir / "TITULO_0"
            titulo_dir.mkdir(parents=True, exist_ok=True)
            out_name = f"titulo_{idx}.txt"   # idx == 0
        else:
            titulo_dir = file_dir / slugify_path(key)
            titulo_dir.mkdir(parents=True, exist_ok=True)
            idx += 1
            out_name = f"titulo_{idx}.txt"

        out_path = titulo_dir / out_name
        out_path.write_text(content, encoding="utf-8")
        saved.append(out_path)
        print(f"  ↳ salvo: {out_path}")

    return saved

# --- Executar split para os arquivos já processados ---
print("\n--- Criando split_docs por TÍTULO (com TÍTULO 0) ---")
for txt_file in arquivos_salvos:
    print(f"--- Dividindo por títulos: {txt_file.name} ---")
    _ = salvar_splits_por_titulo(txt_file, SPLIT_ROOT)

print("✔️ Split finalizado em split_docs/ (preâmbulo preservado como TÍTULO 0)")


--- Criando split_docs por TÍTULO (com TÍTULO 0) ---
--- Dividindo por títulos: L14133.txt ---
  ↳ salvo: data/split_docs/L14133/TITULO_0/titulo_0.txt
  ↳ salvo: data/split_docs/L14133/TITULO_I/titulo_1.txt
  ↳ salvo: data/split_docs/L14133/TITULO_II/titulo_2.txt
  ↳ salvo: data/split_docs/L14133/TITULO_III/titulo_3.txt
  ↳ salvo: data/split_docs/L14133/TITULO_IV/titulo_4.txt
  ↳ salvo: data/split_docs/L14133/TITULO_V/titulo_5.txt
--- Dividindo por títulos: L13709.txt ---
  ↳ salvo: data/split_docs/L13709/TITULO_0/titulo_0.txt
--- Dividindo por títulos: D10024.txt ---
  ↳ salvo: data/split_docs/D10024/TITULO_0/titulo_0.txt
--- Dividindo por títulos: Lcp123.txt ---
  ↳ salvo: data/split_docs/Lcp123/TITULO_0/titulo_0.txt
✔️ Split finalizado em split_docs/ (preâmbulo preservado como TÍTULO 0)


In [4]:
SPLIT_ROOT = Path("data/split_docs")

# --------- slug helpers ---------
def _strip_marker(text: str, marker: str) -> str:
    return text.replace(marker, "").replace("]]", "").strip()

def _slugify(text: str, max_len: int = 120) -> str:
    nfkd = unicodedata.normalize("NFKD", text)
    ascii_text = "".join(ch for ch in nfkd if not unicodedata.combining(ch))
    ascii_text = ascii_text.replace(os.sep, "_").replace("\\", "_").replace("/", "_")
    ascii_text = re.sub(r"[:*?\"<>|]", "_", ascii_text)
    ascii_text = re.sub(r"\s+", " ", ascii_text).strip().replace(" ", "_")
    ascii_text = re.sub(r"_+", "_", ascii_text)
    if len(ascii_text) > max_len:
        ascii_text = ascii_text[:max_len].rstrip("_")
    return ascii_text or "UNTITLED"

def slugify_title_folder(title_marker: str) -> str:
    # title_marker é o texto completo do marcador [[TITLE: ...]]
    base = _strip_marker(title_marker, "[[TITLE:")
    return _slugify(base) or "TITULO"

def slugify_chapter_folder(chapter_marker: str) -> str:
    # chapter_marker é o texto completo do marcador [[CHAPTER: ...]]
    base = _strip_marker(chapter_marker, "[[CHAPTER:")
    return _slugify(base) or "CAPITULO"

# --------- splitters ---------
def split_by_titles_with_preamble(full_text: str) -> List[Tuple[str, str]]:
    """
    Divide exclusivamente por [[TITLE: ...]], preservando o preâmbulo como ('TITULO_0', ...).
    Se não houver [[TITLE: ...]], retorna apenas ('TITULO_0', texto inteiro).
    """
    lines = full_text.splitlines()
    preamble_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []  # (key, lines)
    current_title: str | None = None
    current_lines: List[str] = []
    found_any_title = False

    for line in lines:
        if line.startswith("[[TITLE:"):
            if not found_any_title:
                pre_text = "\n".join(preamble_lines).strip()
                if pre_text:
                    parts.append(("TITULO_0", [pre_text]))
                found_any_title = True
            if current_title is not None:
                parts.append((current_title, current_lines))
            current_title = line.strip()
            current_lines = []
        else:
            if not found_any_title:
                preamble_lines.append(line)
            else:
                if current_title is not None:
                    current_lines.append(line)

    if current_title is not None:
        parts.append((current_title, current_lines))

    if not found_any_title:
        return [("TITULO_0", full_text.strip())]

    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        content = "\n".join(ls).strip()
        normalized.append((key, content))
    return normalized

def split_chapters_with_preamble(title_content: str) -> List[Tuple[str, str]]:
    """
    Dentro do conteúdo de um TÍTULO, divide por [[CHAPTER: ...]].
    - Antes do primeiro capítulo => ('CAPITULO_0', ...)
    - Depois, ('<CHAPTER_MARKER_COMPLETO>', ...), um por capítulo
    Se não houver capítulos, retorna [('CAPITULO_0', title_content)].
    """
    lines = title_content.splitlines()
    pre_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []  # (key, lines)
    current_chapter: str | None = None
    current_lines: List[str] = []
    found_any_chapter = False

    for line in lines:
        if line.startswith("[[CHAPTER:"):
            if not found_any_chapter:
                pre_text = "\n".join(pre_lines).strip()
                parts.append(("CAPITULO_0", [pre_text]))  # pode ser vazio; manter pasta 0
                found_any_chapter = True
            if current_chapter is not None:
                parts.append((current_chapter, current_lines))
            current_chapter = line.strip()
            current_lines = []
        else:
            if not found_any_chapter:
                pre_lines.append(line)
            else:
                if current_chapter is not None:
                    current_lines.append(line)

    if current_chapter is not None:
        parts.append((current_chapter, current_lines))

    if not found_any_chapter:
        return [("CAPITULO_0", title_content.strip())]

    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        content = "\n".join(ls).strip()
        normalized.append((key, content))
    return normalized

# --------- escritor principal ---------
def salvar_splits_por_titulo_e_capitulo(txt_path: Path, split_root: Path = SPLIT_ROOT) -> List[Path]:
    """
    Para cada .txt (já gerado pelo processar_pdf_lei):
      1) Cria data/split_docs/<PDF_STEM>/
      2) Cria pastas de TÍTULO (incluindo TITULO_0) e salva titulo_<idx>.txt
      3) Em cada pasta de TÍTULO, cria 'capitulos/' e subdivide por [[CHAPTER: ...]]:
         - CAPITULO_0/capitulo_0.txt (antes do primeiro capítulo)
         - <PASTA_CAPITULO_K>/capitulo_K.txt para K>=1
    Retorna lista dos caminhos salvos (capítulos e títulos).
    """
    saved: List[Path] = []

    if not txt_path.exists():
        print(f"[split] Arquivo não encontrado: {txt_path}", file=sys.stderr)
        return saved

    split_root.mkdir(parents=True, exist_ok=True)

    # Diretório do arquivo
    file_dir = split_root / txt_path.stem
    if file_dir.exists():
        shutil.rmtree(file_dir)
    file_dir.mkdir(parents=True, exist_ok=True)

    # Lê conteúdo completo
    full_text = txt_path.read_text(encoding="utf-8")

    # --- Split por TÍTULO (com TÍTULO 0) ---
    title_blocks = split_by_titles_with_preamble(full_text)

    title_index = -1  # para TITULO_0 ser 0 ao incrementarmos apenas em títulos "reais"
    for key, title_content in title_blocks:
        if key == "TITULO_0":
            title_folder = file_dir / "TITULO_0"
            title_folder.mkdir(parents=True, exist_ok=True)
            titulo_file = title_folder / "titulo_0.txt"
            titulo_file.write_text(title_content, encoding="utf-8")
            saved.append(titulo_file)
            current_title_idx = 0
        else:
            title_index += 1
            current_title_idx = title_index + 1  # começa em 1
            folder_name = slugify_title_folder(key)
            title_folder = file_dir / folder_name
            title_folder.mkdir(parents=True, exist_ok=True)
            titulo_file = title_folder / f"titulo_{current_title_idx}.txt"
            titulo_file.write_text(title_content, encoding="utf-8")
            saved.append(titulo_file)

        # --- Split por CAPÍTULO dentro do TÍTULO corrente ---
        capitulos_root = title_folder / "capitulos"
        capitulos_root.mkdir(parents=True, exist_ok=True)

        chapter_blocks = split_chapters_with_preamble(title_content)

        # Índice de capítulos reinicia em cada título, com 0 antes do primeiro capítulo
        chapter_idx = -1
        for ch_key, ch_content in chapter_blocks:
            if ch_key == "CAPITULO_0":
                ch_folder = capitulos_root / "CAPITULO_0"
                ch_folder.mkdir(parents=True, exist_ok=True)
                ch_file = ch_folder / "capitulo_0.txt"
                ch_file.write_text(ch_content, encoding="utf-8")
                saved.append(ch_file)
            else:
                chapter_idx += 1
                current_ch_idx = chapter_idx + 1  # 1..n
                ch_folder = capitulos_root / slugify_chapter_folder(ch_key)
                ch_folder.mkdir(parents=True, exist_ok=True)
                ch_file = ch_folder / f"capitulo_{current_ch_idx}.txt"
                ch_file.write_text(ch_content, encoding="utf-8")
                saved.append(ch_file)

        print(f"  ↳ Título {current_title_idx} salvo em: {title_folder}")

    return saved

# --- Executar para os arquivos já processados ---
print("\n--- Criando split de TÍTULOS e CAPÍTULOS (com 0s de preâmbulo) ---")
for txt_file in arquivos_salvos:
    print(f"--- Processando: {txt_file.name} ---")
    _ = salvar_splits_por_titulo_e_capitulo(txt_file, SPLIT_ROOT)

print("✔️ Estrutura criada em data/split_docs/")


--- Criando split de TÍTULOS e CAPÍTULOS (com 0s de preâmbulo) ---
--- Processando: L14133.txt ---
  ↳ Título 0 salvo em: data/split_docs/L14133/TITULO_0
  ↳ Título 1 salvo em: data/split_docs/L14133/TITULO_I
  ↳ Título 2 salvo em: data/split_docs/L14133/TITULO_II
  ↳ Título 3 salvo em: data/split_docs/L14133/TITULO_III
  ↳ Título 4 salvo em: data/split_docs/L14133/TITULO_IV
  ↳ Título 5 salvo em: data/split_docs/L14133/TITULO_V
--- Processando: L13709.txt ---
  ↳ Título 0 salvo em: data/split_docs/L13709/TITULO_0
--- Processando: D10024.txt ---
  ↳ Título 0 salvo em: data/split_docs/D10024/TITULO_0
--- Processando: Lcp123.txt ---
  ↳ Título 0 salvo em: data/split_docs/Lcp123/TITULO_0
✔️ Estrutura criada em data/split_docs/


In [5]:
SPLIT_ROOT = Path("data/split_docs")

# --------- slug helpers ---------
def _strip_marker(text: str, marker: str) -> str:
    return text.replace(marker, "").replace("]]", "").strip()

def _slugify(text: str, max_len: int = 120) -> str:
    nfkd = unicodedata.normalize("NFKD", text)
    ascii_text = "".join(ch for ch in nfkd if not unicodedata.combining(ch))
    ascii_text = ascii_text.replace(os.sep, "_").replace("\\", "_").replace("/", "_")
    ascii_text = re.sub(r"[:*?\"<>|]", "_", ascii_text)
    ascii_text = re.sub(r"\s+", " ", ascii_text).strip().replace(" ", "_")
    ascii_text = re.sub(r"_+", "_", ascii_text)
    if len(ascii_text) > max_len:
        ascii_text = ascii_text[:max_len].rstrip("_")
    return ascii_text or "UNTITLED"

def slugify_title_folder(title_marker: str) -> str:
    base = _strip_marker(title_marker, "[[TITLE:")
    return _slugify(base) or "TITULO"

def slugify_chapter_folder(chapter_marker: str) -> str:
    base = _strip_marker(chapter_marker, "[[CHAPTER:")
    return _slugify(base) or "CAPITULO"

def article_id_from_marker(article_marker: str) -> str:
    """
    Extrai o ID do artigo do marcador [[ARTICLE: <id>]] e normaliza para uso no nome do arquivo.
    Ex.: '5', '5-A', '37B' -> mantém hífen/letras; remove espaços.
    """
    base = _strip_marker(article_marker, "[[ARTICLE:")
    base = base.replace(" ", "")
    # Permitir apenas [A-Za-z0-9_-] para garantir nome de arquivo seguro
    base = re.sub(r"[^A-Za-z0-9_-]", "", base)
    return base or "X"

# --------- splitters ---------
def split_by_titles_with_preamble(full_text: str) -> List[Tuple[str, str]]:
    """[[TITLE: ...]] com preâmbulo como ('TITULO_0', ...)."""
    lines = full_text.splitlines()
    preamble_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []
    current_title: str | None = None
    current_lines: List[str] = []
    found_any_title = False

    for line in lines:
        if line.startswith("[[TITLE:"):
            if not found_any_title:
                pre_text = "\n".join(preamble_lines).strip()
                if pre_text or True:  # cria sempre TITULO_0 (pode ser vazio)
                    parts.append(("TITULO_0", [pre_text]))
                found_any_title = True
            if current_title is not None:
                parts.append((current_title, current_lines))
            current_title = line.strip()
            current_lines = []
        else:
            if not found_any_title:
                preamble_lines.append(line)
            else:
                if current_title is not None:
                    current_lines.append(line)

    if current_title is not None:
        parts.append((current_title, current_lines))

    if not found_any_title:
        return [("TITULO_0", full_text.strip())]

    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        content = "\n".join(ls).strip()
        normalized.append((key, content))
    return normalized

def split_chapters_with_preamble(title_content: str) -> List[Tuple[str, str]]:
    """Dentro de um TÍTULO, divide por [[CHAPTER: ...]] com CAPITULO_0 como preâmbulo."""
    lines = title_content.splitlines()
    pre_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []
    current_chapter: str | None = None
    current_lines: List[str] = []
    found_any_chapter = False

    for line in lines:
        if line.startswith("[[CHAPTER:"):
            if not found_any_chapter:
                pre_text = "\n".join(pre_lines).strip()
                parts.append(("CAPITULO_0", [pre_text]))  # preâmbulo (pode ser vazio)
                found_any_chapter = True
            if current_chapter is not None:
                parts.append((current_chapter, current_lines))
            current_chapter = line.strip()
            current_lines = []
        else:
            if not found_any_chapter:
                pre_lines.append(line)
            else:
                if current_chapter is not None:
                    current_lines.append(line)

    if current_chapter is not None:
        parts.append((current_chapter, current_lines))

    if not found_any_chapter:
        return [("CAPITULO_0", title_content.strip())]

    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        content = "\n".join(ls).strip()
        normalized.append((key, content))
    return normalized

def split_articles_with_preamble(chapter_content: str) -> List[Tuple[str, str]]:
    """
    Dentro do conteúdo de um CAPÍTULO, divide por [[ARTICLE: <id>]].
    - Antes do primeiro artigo => ('ARTIGO_0', ...)
    - Cada artigo => ('[[ARTICLE: <id>]]', conteúdo até o próximo)
    Se não houver artigos, retorna [('ARTIGO_0', chapter_content)].
    """
    lines = chapter_content.splitlines()
    pre_lines: List[str] = []
    parts: List[Tuple[str, List[str]]] = []
    current_article: str | None = None
    current_lines: List[str] = []
    found_any_article = False

    for line in lines:
        if line.startswith("[[ARTICLE:"):
            if not found_any_article:
                pre_text = "\n".join(pre_lines).strip()
                parts.append(("ARTIGO_0", [pre_text]))  # preâmbulo (pode ser vazio)
                found_any_article = True
            if current_article is not None:
                parts.append((current_article, current_lines))
            current_article = line.strip()
            current_lines = []
        else:
            if not found_any_article:
                pre_lines.append(line)
            else:
                if current_article is not None:
                    current_lines.append(line)

    if current_article is not None:
        parts.append((current_article, current_lines))

    if not found_any_article:
        return [("ARTIGO_0", chapter_content.strip())]

    normalized: List[Tuple[str, str]] = []
    for key, ls in parts:
        content = "\n".join(ls).strip()
        normalized.append((key, content))
    return normalized

# --------- escritor principal ---------
def salvar_splits_por_titulo_capitulo_artigos(txt_path: Path, split_root: Path = SPLIT_ROOT) -> List[Path]:
    """
    Para cada .txt (gerado por processar_pdf_lei):
      1) Cria data/split_docs/<PDF_STEM>/
      2) Cria pastas de TÍTULO (incluindo TITULO_0) e salva titulo_<idx>.txt
      3) Em cada TÍTULO, cria 'capitulos/' e subdivide por [[CHAPTER: ...]]:
         - CAPITULO_0/capitulo_0.txt (preâmbulo)
         - <PASTA_CAPITULO_K>/capitulo_K.txt (K>=1)
      4) Em cada CAPÍTULO, cria 'artigos/' e subdivide por [[ARTICLE: <id>]]:
         - artigos/artigo_0.txt (antes do primeiro artigo)
         - artigos/artigo_<ID>.txt (um por artigo, com <ID> do marcador)
    """
    saved: List[Path] = []

    if not txt_path.exists():
        print(f"[split] Arquivo não encontrado: {txt_path}", file=sys.stderr)
        return saved

    split_root.mkdir(parents=True, exist_ok=True)

    # Diretório do arquivo
    file_dir = split_root / txt_path.stem
    if file_dir.exists():
        shutil.rmtree(file_dir)
    file_dir.mkdir(parents=True, exist_ok=True)

    # Lê conteúdo completo
    full_text = txt_path.read_text(encoding="utf-8")

    # --- Split por TÍTULO (com TÍTULO 0) ---
    title_blocks = split_by_titles_with_preamble(full_text)

    title_index = -1  # para TITULO_0 ser 0
    for key, title_content in title_blocks:
        if key == "TITULO_0":
            title_folder = file_dir / "TITULO_0"
            title_folder.mkdir(parents=True, exist_ok=True)
            titulo_file = title_folder / "titulo_0.txt"
            titulo_file.write_text(title_content, encoding="utf-8")
            saved.append(titulo_file)
            current_title_idx = 0
        else:
            title_index += 1
            current_title_idx = title_index + 1  # 1..n
            folder_name = slugify_title_folder(key)
            title_folder = file_dir / folder_name
            title_folder.mkdir(parents=True, exist_ok=True)
            titulo_file = title_folder / f"titulo_{current_title_idx}.txt"
            titulo_file.write_text(title_content, encoding="utf-8")
            saved.append(titulo_file)

        # --- Split por CAPÍTULO dentro do TÍTULO corrente ---
        capitulos_root = title_folder / "capitulos"
        capitulos_root.mkdir(parents=True, exist_ok=True)

        chapter_blocks = split_chapters_with_preamble(title_content)

        chapter_idx = -1
        for ch_key, ch_content in chapter_blocks:
            if ch_key == "CAPITULO_0":
                ch_folder = capitulos_root / "CAPITULO_0"
                ch_folder.mkdir(parents=True, exist_ok=True)
                ch_file = ch_folder / "capitulo_0.txt"
                ch_file.write_text(ch_content, encoding="utf-8")
                saved.append(ch_file)
                target_chapter_folder = ch_folder
            else:
                chapter_idx += 1
                current_ch_idx = chapter_idx + 1  # 1..n
                ch_folder = capitulos_root / slugify_chapter_folder(ch_key)
                ch_folder.mkdir(parents=True, exist_ok=True)
                ch_file = ch_folder / f"capitulo_{current_ch_idx}.txt"
                ch_file.write_text(ch_content, encoding="utf-8")
                saved.append(ch_file)
                target_chapter_folder = ch_folder

            # --- Split por ARTIGOS dentro do CAPÍTULO corrente ---
            artigos_root = target_chapter_folder / "artigos"
            artigos_root.mkdir(parents=True, exist_ok=True)

            article_blocks = split_articles_with_preamble(ch_content)

            for a_key, a_content in article_blocks:
                if a_key == "ARTIGO_0":
                    a_file = artigos_root / "artigo_0.txt"
                else:
                    art_id = article_id_from_marker(a_key)
                    a_file = artigos_root / f"artigo_{art_id}.txt"
                a_file.write_text(a_content, encoding="utf-8")
                saved.append(a_file)

        print(f"  ↳ Título {current_title_idx} salvo em: {title_folder}")

    return saved

# --- Executar para os arquivos já processados ---
print("\n--- Criando split de TÍTULOS, CAPÍTULOS e ARTIGOS (com 0s de preâmbulo) ---")
for txt_file in arquivos_salvos:
    print(f"--- Processando: {txt_file.name} ---")
    _ = salvar_splits_por_titulo_capitulo_artigos(txt_file, SPLIT_ROOT)

print("✔️ Estrutura criada em data/split_docs/ (inclui artigos/ com artigo_0 e artigo_<ID>.txt)")



--- Criando split de TÍTULOS, CAPÍTULOS e ARTIGOS (com 0s de preâmbulo) ---
--- Processando: L14133.txt ---
  ↳ Título 0 salvo em: data/split_docs/L14133/TITULO_0
  ↳ Título 1 salvo em: data/split_docs/L14133/TITULO_I
  ↳ Título 2 salvo em: data/split_docs/L14133/TITULO_II
  ↳ Título 3 salvo em: data/split_docs/L14133/TITULO_III
  ↳ Título 4 salvo em: data/split_docs/L14133/TITULO_IV
  ↳ Título 5 salvo em: data/split_docs/L14133/TITULO_V
--- Processando: L13709.txt ---
  ↳ Título 0 salvo em: data/split_docs/L13709/TITULO_0
--- Processando: D10024.txt ---
  ↳ Título 0 salvo em: data/split_docs/D10024/TITULO_0
--- Processando: Lcp123.txt ---
  ↳ Título 0 salvo em: data/split_docs/Lcp123/TITULO_0
✔️ Estrutura criada em data/split_docs/ (inclui artigos/ com artigo_0 e artigo_<ID>.txt)


# Dataset de artigos

In [6]:
SPLIT_ROOT = Path("data/split_docs")

paths = [p.relative_to(SPLIT_ROOT).as_posix()
         for p in SPLIT_ROOT.rglob("artigos/*.txt")]

paths = sorted(paths)

rows = []
for s in paths:
    parts = s.split('/')  # [lei, titulo, 'capitulos', capitulo, 'artigos', artigo.txt]
    if len(parts) < 6:
        continue
    # lê o conteúdo do arquivo
    txt_path = SPLIT_ROOT / s
    texto = txt_path.read_text(encoding='utf-8', errors='ignore')

    rows.append({
        'lei': parts[0],
        'titulo': parts[1],
        'capitulo': parts[3],
        'artigo': parts[5],   # 'artigo_2.txt' etc.
        'path': s,            # caminho após split_docs
        'texto': texto,       # conteúdo do arquivo
    })

df = pd.DataFrame(rows, columns=['lei', 'titulo', 'capitulo', 'artigo', 'path', 'texto'])
df

,lei,titulo,capitulo,artigo,path,texto
0,D10024,TITULO_0,CAPITULO_0,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_0/artigos/a...,Presidência da República\nSecretaria-Geral\nSu...
1,D10024,TITULO_0,CAPITULO_I,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,DISPOSIÇÕES PRELIMINARES\nObjeto e âmbito de a...
2,D10024,TITULO_0,CAPITULO_I,artigo_1.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 1º Este Decreto regulamenta a licitação, ..."
3,D10024,TITULO_0,CAPITULO_I,artigo_2.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 2º O pregão, na forma eletrônica, é condi..."
4,D10024,TITULO_0,CAPITULO_I,artigo_3.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 3º Para fins do disposto neste Decreto, c..."
...,...,...,...,...,...,...
569,Lcp123,TITULO_0,CAPITULO_XIV,artigo_86.txt,Lcp123/TITULO_0/capitulos/CAPITULO_XIV/artigos...,Art. 86. As matérias tratadas nesta Lei Comple...
570,Lcp123,TITULO_0,CAPITULO_XIV,artigo_87-A.txt,Lcp123/TITULO_0/capitulos/CAPITULO_XIV/artigos...,"Art. 87-A. Os Poderes Executivos da União, Est..."
571,Lcp123,TITULO_0,CAPITULO_XIV,artigo_87.txt,Lcp123/TITULO_0/capitulos/CAPITULO_XIV/artigos...,Art. 87. O § 1º do art. 3º da Lei Complementar...
572,Lcp123,TITULO_0,CAPITULO_XIV,artigo_88.txt,Lcp123/TITULO_0/capitulos/CAPITULO_XIV/artigos...,Art. 88. Esta Lei Complementar entra em vigor ...


In [7]:
MODEL_ID = "gemini-2.5-flash-lite"  # use exatamente o modelo que você vai chamar
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

_model = genai.GenerativeModel(MODEL_ID)

def contar_tokens_gemini(texto: str) -> int:
    if not texto:
        return 0
    resp = _model.count_tokens(texto)
    # resp.total_tokens existe nas versões recentes; int() para garantir tipo
    return int(getattr(resp, "total_tokens", 0))

# adiciona a coluna
df["tokens"] = df["texto"].progress_apply(contar_tokens_gemini)
df.head()


100%|██████████| 574/574 [01:56<00:00,  4.92it/s]


,lei,titulo,capitulo,artigo,path,texto,tokens
0,D10024,TITULO_0,CAPITULO_0,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_0/artigos/a...,Presidência da República\nSecretaria-Geral\nSu...,239
1,D10024,TITULO_0,CAPITULO_I,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,DISPOSIÇÕES PRELIMINARES\nObjeto e âmbito de a...,15
2,D10024,TITULO_0,CAPITULO_I,artigo_1.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 1º Este Decreto regulamenta a licitação, ...",461
3,D10024,TITULO_0,CAPITULO_I,artigo_2.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 2º O pregão, na forma eletrônica, é condi...",227
4,D10024,TITULO_0,CAPITULO_I,artigo_3.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 3º Para fins do disposto neste Decreto, c...",946


In [8]:
df.describe()

,tokens
count,574.000000
mean,343.698606
std,947.902415
min,4.000000
25%,57.000000
50%,136.500000
75%,364.500000
max,16130.000000


# Save

In [9]:
os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/v1_processed_articles.csv", index=False)